In [ ]:
import math
from random import randint
import random
import matplotlib.pyplot as plt
import numpy as np

#Objective Function number 1
def calculate_PCSI(V_Car, V_Obstcale, D, theta):

    PCSI_Delta_V = (V_Car - V_Obstcale) / D  #V is the velocity and D is the distance between the 2 vechiles or the vechile and the obstcale

    #theta is the angle between the 2 vechiles
    if 0 <= theta <= 15 or 165 <= theta <= 180:
        PCSI_theta = 1
    elif 15 <= theta <= 45 or 135 <= theta <= 165:
        PCSI_theta = 3
    elif 45 <= theta <= 90 or 90 <= theta <= 135:
        PCSI_theta = 2
    else:
        PCSI_theta = 0

    PCSI_mass_ratio = 1 #the mass ratio between the 2 vechiles or the vechile and the obstcale

    total_PCSI = (1/3) * PCSI_Delta_V + (1/3) * PCSI_theta + (1/3) * PCSI_mass_ratio

    return total_PCSI


#Objective Function number 2
def calculate_CSI(length, width):

  e = 0.5 #the coefficient of restitution
  m1 = 1000 #the mass of the vechile
  m2 = 1000 #the mass of the obstcale
  Rk = 1 #the stiffness ratio
  Rm = 1 #the mass ratio
  p1 = 100
  p2 = math.sqrt((m2 * (((3.8 **2) / 12) + ((1.5 ** 2 ) / 12))) / m2)
  h2 = 0.25 * 3.8 #Force Arm for the obstcale
  h1 = 0.25 * length #Force Arm for the vechile
  p1 = math.sqrt((m1 * (((length **2) / 12) + ((width ** 2 ) / 12))) / m1) #the radius of gyration for the vechile

  Y1 = (p1**2) / ((p1**2) + (h1**2)) #the radius of gyration for the vechile
  Y2 = (p2**2) / ((p2**2) + (h2**2)) #the radius of gyration for the obstcale



  total_CSI = math.sqrt(( Y1 * Y2 * (1- (e**2) ) ) / ((1 + Rk)  * (Y2 + (Y1 * Rm))))

  return total_CSI




# V_Car = 70
# V_Obstcale = 40
# theta = 30
# D = 2

# PCSI_result = calculate_PCSI(V_Car, V_Obstcale ,D, theta)
# print(f"The Potential Crash Severity Index (PCSI) is: {PCSI_result}")

# e = 0.8
# Rk = 0.9
# Rm = 0.95
# h = 0.25

#CSI_result = calculate_CSI()
#print(f"The Crash Severity Index (CSI) is: {CSI_result}")

In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np

#arrays to store the changing in values for the decision variables and the objective function
f_best_values = []
f_curr_values = []
v_best_values = []
v_curr_values = []
theta_best_values = []
theta_curr_values = []

#The SA for the first objective function
def SA(V_Car, V_Obstcale, D, theta):

  #initialization of the values
  V_Car_curr = V_Car
  theta_curr = theta

  i_max = 100;
  T0 = 1000;
  Tf = 0.01
  nt = 1;
  B = (Tf - T0) / i_max

  xs_V = V_Car #xs_V is the best value for V
  xs_theta = theta #xs_theta is the best value for theta
  fs = calculate_PCSI(V_Car, V_Obstcale, D, theta)
  f_old = fs

  T_Curr = T0
  i_Curr = 1
  r = 0

  #calculating the maximum reduction possible in the velocity of the car
  Time_to_Impact = D / (V_Car - V_Obstcale)
  V_Car_Min = (-8 * Time_to_Impact) + V_Car
  V_Reduction_Max = V_Car - V_Car_Min

  while ((T_Curr > Tf) & (i_Curr < i_max)) :

      theta_new =  theta + randint(-40, 40)
      V_Car_new =  random.uniform(V_Car, V_Car - V_Reduction_Max)

      f_new = calculate_PCSI(V_Car_new, V_Obstcale, D, theta_new)
      delta_E = f_new - f_old

      r = random.uniform(0,1)

      if (delta_E < 0):
          V_Car_curr = V_Car_new
          theta_curr = theta_new
          f_old = f_new

      elif ((2.71828 ** (-delta_E / T_Curr)) > r):
          V_Car_curr = V_Car_new
          theta_curr = theta_new
          f_old = f_new


      if (f_new < fs):
           xs_V = V_Car_curr
           xs_theta = theta_curr
           fs = f_new

      #getting the current and best value till now for the decision vars and the objective function
      f_curr_values.append(f_old)
      f_best_values.append(fs)
      v_curr_values.append(V_Car_curr)
      v_best_values.append(xs_V)
      theta_curr_values.append(theta_curr)
      theta_best_values.append(xs_theta)

      i_Curr = i_Curr + 1
      T_Curr = T0 - (B * i_Curr)

  return xs_V, xs_theta, fs

V_Car = 70
V_Obstcale = -30
D = 50
theta = 140

SA_Result = SA(V_Car, V_Obstcale, D, theta)
#print(SA_Result)
print(f"The best V is {SA_Result[0]}")
print(f"The best theta is {SA_Result[1]}")
print(f"The best objective function result is {SA_Result[2]}")

#Plotting the graphs

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(f_curr_values)
plt.xlabel("Iteration")
plt.ylabel("Objective Function (PCSI)")
plt.title("Objective Function (PCSI) vs. Iteration")

plt.subplot(1, 2, 2)
plt.plot(f_best_values)
plt.xlabel("Iteration")
plt.ylabel("Best F")
plt.title("Best Function vs. Iteration")

plt.tight_layout()
plt.show()

print()

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(v_curr_values)
plt.xlabel("Iteration")
plt.ylabel("Current Velocity")
plt.title("Current Velocity vs. Iteration")

plt.subplot(1, 2, 2)
plt.plot(v_best_values)
plt.xlabel("Iteration")
plt.ylabel("Best Velocity")
plt.title("Best Velocity vs. Iteration")

plt.tight_layout()
plt.show()

print()

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(theta_curr_values)
plt.xlabel("Iteration")
plt.ylabel("Current Theta")
plt.title("Current Theta vs. Iteration")

plt.subplot(1, 2, 2)
plt.plot(theta_best_values)
plt.xlabel("Iteration")
plt.ylabel("Best Theta")
plt.title("Best Theta vs. Iteration")

plt.tight_layout()
plt.show()

In [ ]:
#arrays to store the changing in values for the decision variables and the objective function
f_best_values = []
f_curr_values = []
l_best_values = []
l_curr_values = []
w_best_values = []
w_curr_values = []

#The SA for the first objective function
def SA2():

  #initialization of the variables
  length = random.uniform(3.8, 4.9)
  width = random.uniform(1.5, 2)

  i_max = 100;
  T0 = 1000;
  Tf = 0.01
  nt = 1;
  B = (Tf - T0) / i_max

  fs = calculate_CSI(length, width)
  ls = length
  ws = width

  l_curr = length
  w_curr = width
  f_curr = fs


  T_Curr = T0
  i_Curr = 1
  r = 0

  while ((T_Curr > Tf) & (i_Curr < i_max)) :

    l_new = random.uniform(3.8, 4.9)
    w_new = random.uniform(1.5, 2)
    f_new = calculate_CSI(l_new, w_new)

    delta_E = f_new - f_curr
    r = random.uniform(0,1)

    if (delta_E < 0):
      l_curr = l_new
      w_curr = w_new
      f_curr = f_new

    elif ((e ** (-delta_E / T_Curr)) > r):
      l_curr = l_new
      w_curr = w_new
      f_curr = f_new


    if (f_new < fs):
       ls = l_new
       ws = w_new
       fs = f_new

    #getting the current and best value till now for the decision vars and the objective function
    f_curr_values.append(f_curr)
    f_best_values.append(fs)
    l_curr_values.append(l_curr)
    l_best_values.append(ls)
    w_curr_values.append(w_curr)
    w_best_values.append(ws)

    i_Curr = i_Curr + 1
    T_Curr = T0 - (B * i_Curr)

  return ls, ws, fs



SA2_Result = SA2()
#print(SA2_Result)
print(f"The best length is {SA2_Result[0]}")
print(f"The best width is {SA2_Result[1]}")
print(f"The best objective function result is {SA2_Result[2]}")

#Plotting the graphs

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(f_curr_values)
plt.xlabel("Iteration")
plt.ylabel("Objective Function (CSI)")
plt.title("Objective Function (CSI) vs. Iteration")

plt.subplot(1, 2, 2)
plt.plot(f_best_values)
plt.xlabel("Iteration")
plt.ylabel("Best Function")
plt.title("Best Function vs. Iteration")

plt.tight_layout()
plt.show()

print()

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(l_curr_values)
plt.xlabel("Iteration")
plt.ylabel("Current Length (CSI)")
plt.title("Current Length (CSI) vs. Iteration")

plt.subplot(1, 2, 2)
plt.plot(l_best_values)
plt.xlabel("Iteration")
plt.ylabel("Best Length")
plt.title("Best Length vs. Iteration")

plt.tight_layout()
plt.show()

print()

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(w_curr_values)
plt.xlabel("Iteration")
plt.ylabel("Current Width (CSI)")
plt.title("Current Width (CSI) vs. Iteration")

plt.subplot(1, 2, 2)
plt.plot(w_best_values)
plt.xlabel("Iteration")
plt.ylabel("Best Width")
plt.title("Best Width vs. Iteration")

plt.tight_layout()
plt.show()